[Rendered Notebook](http://nbviewer.jupyter.org/github/andersonfrailey/Notebook-Uploads/blob/master/2011%20PUF%20Comparisons.ipynb)

In [141]:
from taxcalc import *
from taxcalc.utils import *
from bokeh.io import show, output_notebook
from bkcharts import Bar, Scatter
from bokeh.layouts import column
from bokeh.plotting import figure
from bokeh.models import HoverTool
from collections import OrderedDict
import copy
import pandas as pd
from notebookfunctions import distribution, index_list, percentile
output_notebook()

Loading BokehJS ...

In [142]:
# Data from IRS-SOI Tax Stats. Used for comparison
soi_stats = pd.read_csv('soi_stats.csv', index_col=0)  # Equivalent to tax-calc diagnostic table
soi_income = pd.read_csv('soi_income_stats.csv')  # Distribution of income items
soi_deductions = pd.read_csv('soi_deductions.csv', index_col=0)  # Itemized deductions
soi_deductions['index'] = soi_deductions.index

In [143]:
# Read in new PUF and associated data
gf = pd.read_csv('../taxdata/stage1/growfactors.csv')
wt = pd.read_csv('../taxdata/puf_stage2/puf_weights.csv')
puf = pd.read_csv('../taxdata/puf_data/puf.csv')
adj = pd.read_csv('../taxdata/puf_stage3/puf_ratios.csv', index_col=0)
adj = adj.transpose()

In [144]:
# base calculator
calc = Calculator(records=Records(), policy = Policy())
calc.advance_to_year(2014)
calc.calc_all()

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [145]:
# new calculator
growf = Growfactors('../taxdata/stage1/growfactors.csv')
rec = Records(puf, gfactors=growf,
              weights=wt, adjust_ratios=adj,
              start_year=2011)
pol = Policy(gfactors=growf, start_year=2011, num_years=16)
consump = Consumption(start_year=2011, num_years=16)
behave = Behavior(start_year=2011, num_years=16)
calc_new = Calculator(records=rec, policy=pol, consumption=consump, behavior=behave)
calc_new.advance_to_year(2014)
calc_new.calc_all()

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2011.


### New Distribution Table 

In [146]:
create_distribution_table(calc_new.records, groupby='weighted_deciles',
                          result_type='weighted_sum', income_measure='c00100')

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"16,128,246","-82,136,288,688","4,523,604","121,312,094,858",0,0,"85,195,484,181",0,0,"-72,638,646,894","8,876","2,368,796,593","2,368,796,593","3,490,682","60,284,492","763,050,141","1,662,540,262","2,394,106,954","4,056,647,216"
1,"16,130,008","59,997,722,018","16,101,999","107,230,891,193","18,869","77,672,150","66,556,948,522","678,801,561","30,724,302","59,947,804,587",619,"83,041,579","113,765,881","288,508","31,829,399","4,098,844,533","-3,953,537,761","6,749,521,635","2,795,983,873"
2,"16,130,574","156,203,652,446","15,942,056","127,350,614,487","183,353","1,870,085,118","94,104,191,583","8,690,466,599","839,426,057","155,650,392,006","14,984","146,256,867","985,682,923","44,476,707","91,698,615","17,834,065,168","-16,801,160,336","17,710,290,063","909,129,727"
3,"16,129,194","260,243,460,711","15,221,474","130,717,686,368","907,081","12,129,307,521","118,001,613,645","46,500,666,974","4,504,576,530","251,841,668,594","35,519","113,651,259","4,618,227,789","659,455,940","121,179,119","30,437,699,482","-26,357,748,513","31,323,349,171","4,965,600,657"
4,"16,130,248","392,908,703,884","14,511,600","126,697,628,878","1,615,749","24,151,058,990","121,633,998,138","133,337,125,005","14,687,969,985","377,693,032,973","26,861","230,033,193","14,918,003,178","2,845,889,837","260,901,752","24,150,658,612","-11,817,643,518","49,737,654,233","37,920,010,714"
5,"16,128,247","560,434,623,875","13,009,877","115,624,301,301","3,116,537","47,491,916,254","123,051,123,432","277,730,195,558","33,300,401,771","531,674,285,834","25,563","221,492,309","33,521,894,080","5,945,681,533","417,816,683","13,624,820,457","14,369,208,773","71,149,258,903","85,518,467,677"
6,"16,130,942","783,379,086,983","10,908,216","105,131,671,864","5,219,128","86,198,060,966","130,698,376,686","464,125,429,554","59,316,792,259","732,537,999,842","18,674","333,778,397","59,650,570,656","9,075,141,423","645,354,706","3,944,338,593","47,276,445,346","97,807,411,778","145,083,857,124"
7,"16,128,936","1,106,724,218,753","8,537,278","93,526,790,512","7,589,254","139,840,193,706","146,115,131,472","728,187,877,499","103,738,981,161","1,026,616,813,323","40,084","429,821,545","104,168,802,706","11,969,131,525","989,426,893","1,362,113,862","91,826,984,212","132,572,876,556","224,399,860,768"
8,"16,130,336","1,635,555,753,497","5,480,263","66,946,848,483","10,649,867","233,090,741,232","168,872,770,458","1,167,613,389,321","177,811,222,653","1,504,939,928,611","122,017","489,078,417","178,300,301,070","14,192,595,425","1,493,927,026","1,467,831,216","164,133,801,456","200,917,979,743","365,051,781,198"
9,"16,129,868","4,946,341,168,385","1,803,000","22,134,027,126","14,326,178","614,385,853,901","160,753,680,422","4,150,893,289,075","1,029,292,997,320","4,650,225,136,205","3,947,098","29,817,497,843","1,059,110,495,162","26,216,402,322","29,051,263,487","858,519,426","1,061,086,836,901","348,002,655,976","1,409,089,492,878"


### Current Distribution Table

In [147]:
create_distribution_table(calc.records, groupby='weighted_deciles',
                          result_type='weighted_sum', income_measure='c00100')

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"16,221,180","-149,460,064,489","3,744,863","123,575,783,463",0,0,"86,517,192,426",0,0,"-136,551,737,321","11,482","3,185,179,873","3,185,179,873","3,536,001","86,953,480","616,578,863","2,652,018,489","2,655,084,698","5,307,103,187"
1,"16,223,729","49,184,455,359","16,194,438","102,686,835,811","16,187","54,448,503","62,846,564,468","707,896,323","30,972,273","49,521,983,383",910,"138,962,678","169,934,951","469,458","25,782,677","3,103,722,273","-2,908,474,104","5,564,415,908","2,655,941,804"
2,"16,222,054","143,025,181,773","16,084,032","125,766,068,153","127,876","1,494,700,096","88,687,706,918","6,577,949,758","616,897,355","142,231,061,802","21,501","73,252,389","690,149,744","24,143,818","96,283,084","16,028,317,354","-15,266,028,344","16,098,662,596","832,634,253"
3,"16,222,061","249,007,117,256","15,445,471","130,017,769,754","776,589","10,064,110,445","114,950,213,802","42,072,932,766","4,082,797,316","242,532,927,905","42,662","218,792,743","4,301,590,060","565,876,732","116,675,080","31,138,615,686","-27,286,227,278","29,903,096,954","2,616,869,676"
4,"16,223,077","382,237,480,535","14,707,911","127,434,041,161","1,514,122","23,226,423,831","121,845,956,204","124,522,197,022","13,595,505,252","367,840,244,013","29,678","193,827,015","13,789,332,267","2,751,571,523","250,755,552","25,343,769,445","-14,055,253,148","47,519,575,469","33,464,322,321"
5,"16,222,364","553,683,615,585","13,422,207","118,411,365,556","2,800,097","43,366,254,480","123,996,988,784","271,773,957,371","32,445,431,882","527,668,459,601","19,027","192,316,962","32,637,748,844","6,171,858,258","442,915,714","14,237,656,793","12,671,149,507","69,742,633,559","82,413,783,066"
6,"16,222,462","784,673,207,701","11,244,737","104,719,291,632","4,974,603","81,894,183,170","125,246,634,800","475,540,570,351","61,500,322,019","738,410,698,876","19,309","355,178,754","61,855,500,773","9,102,329,772","724,292,445","3,605,478,869","49,871,984,577","95,817,129,261","145,689,113,838"
7,"16,222,325","1,112,628,091,949","8,987,298","94,849,287,654","7,233,119","131,238,041,801","139,340,036,194","748,355,843,455","108,801,811,258","1,042,158,310,339","64,228","593,218,652","109,395,029,911","11,756,929,692","1,043,395,654","1,386,894,462","97,294,601,411","132,016,453,995","229,311,055,405"
8,"16,220,906","1,667,906,688,861","5,896,284","69,961,594,639","10,322,867","222,365,935,945","164,636,367,745","1,211,715,364,476","189,432,031,965","1,551,088,910,222","187,059","709,967,053","190,141,999,018","14,428,285,987","1,604,554,656","1,532,935,494","175,785,332,194","202,957,438,570","378,742,770,763"
9,"16,224,463","5,228,629,819,438","2,015,937","24,213,324,443","14,207,685","591,442,837,851","154,871,714,225","4,460,532,934,577","1,123,648,784,101","4,960,990,883,398","4,407,703","33,017,276,304","1,156,666,060,405","28,941,930,460","34,766,511,097","870,959,970","1,161,619,681,071","354,326,705,715","1,515,946,386,786"


### Diagnostic Table Comparison

In [148]:
new_diag = create_diagnostic_table(calc_new)
diag = create_diagnostic_table(calc)  # Current PUF

In [149]:
diag_data = pd.DataFrame()
diag_data['SOI'] = soi_stats['Value']
diag_data['New'] = new_diag[2014]
diag_data['Current'] = diag[2014]
diag_data['% Change'] = ((new_diag[2014] / diag[2014]) - 1) * 100
diag_data['New - SOI'] = map(abs, new_diag[2014] - diag_data['SOI'])
diag_data['Current - SOI'] = map(abs, diag[2014] - diag_data['SOI'])
diag_data

,SOI,New,Current,% Change,New - SOI,Current - SOI
Returns (#m),148.6,161.3,162.2,-0.6,12.7,13.6
AGI ($b),"9,771.0","9,819.7","10,021.5",-2.0,48.7,250.5
Itemizers (#m),44.0,43.6,42.0,3.9,0.4,2.0
Itemized Deduction ($b),"1,206.7","1,159.3","1,105.2",4.9,47.4,101.5
Standard Deduction Filers (#m),117.4,106.0,107.7,-1.6,11.4,9.7
Standard Deduction ($b),876.2,926.3,925.2,0.1,50.1,49.0
Personal Exemption ($b),"1,121.6","1,150.0","1,114.0",3.2,28.4,7.6
Taxable Income ($b),"6,997.9","6,977.8","7,341.8",-5.0,20.1,343.9
Regular Tax ($b),nan,"1,423.5","1,534.2",-7.2,nan,nan
AMT Income ($b),nan,"9,218.5","9,485.9",-2.8,nan,nan


### Income Levels 

In [150]:
inc_dict = OrderedDict()
inc_dict['New'] = [] 
inc_dict['Current'] = []
inc_dict['SOI'] = []
inc_dict['Pct Diff'] = []
inc_dict['New - SOI'] = []
inc_dict['Current - SOI'] = []
inc_list = ['WAS', 'Taxable Interest', 'Ordinary Dividends', 'Qualified Dividends', 'Business Income']
# Wage and salary
was_new = (calc_new.records.e00200 * calc_new.records.s006).sum()
inc_dict['New'].append(was_new)
was = (calc.records.e00200 * calc.records.s006).sum()
inc_dict['Current'].append(was)
soi_was = soi_income['WAS'].sum()
inc_dict['SOI'].append(soi_was)
new_was_diff = abs(was_new - soi_was)
inc_dict['New - SOI'].append(new_was_diff)
curr_was_diff = abs(was - soi_was)
inc_dict['Current - SOI'].append(curr_was_diff)
inc_dict['Pct Diff'].append((curr_was_diff / was) * 100)
# Interest income
int_new = (calc_new.records.e00300 * calc_new.records.s006).sum()
inc_dict['New'].append(int_new)
int_ = (calc.records.e00300 * calc.records.s006).sum()
inc_dict['Current'].append(int_)
int_soi = soi_income['INT'].sum()
inc_dict['SOI'].append(int_soi)
new_int_diff = abs(int_new - int_soi)
inc_dict['New - SOI'].append(new_int_diff)
curr_int_diff = abs(int_ - int_soi)
inc_dict['Current - SOI'].append(curr_int_diff)
inc_dict['Pct Diff'].append((curr_int_diff / int_) * 100)
# Ordinary dividends
odiv_new = (calc_new.records.e00600 * calc_new.records.s006).sum()
inc_dict['New'].append(odiv_new)
odiv = (calc.records.e00600 * calc.records.s006).sum()
inc_dict['Current'].append(odiv)
odiv_soi = soi_income['ODIV'].sum()
inc_dict['SOI'].append(odiv_soi)
new_odiv_diff = abs(odiv_new - odiv_soi)
inc_dict['New - SOI'].append(new_odiv_diff)
curr_odiv_diff = abs(odiv - odiv_soi)
inc_dict['Current - SOI'].append(curr_odiv_diff)
inc_dict['Pct Diff'].append((curr_odiv_diff / odiv) * 100)
# Qualified dividends
qdiv_new = (calc_new.records.e00650 * calc_new.records.s006).sum()
inc_dict['New'].append(qdiv_new)
qdiv = (calc.records.e00650 * calc.records.s006).sum()
inc_dict['Current'].append(qdiv)
qdiv_soi = soi_income['QDIV'].sum()
inc_dict['SOI'].append(qdiv_soi)
new_qdiv_diff = abs(qdiv_new - qdiv_soi)
inc_dict['New - SOI'].append(new_qdiv_diff)
curr_qdiv_diff = abs(qdiv - qdiv_soi)
inc_dict['Current - SOI'].append(curr_qdiv_diff)
inc_dict['Pct Diff'].append((curr_qdiv_diff / qdiv) * 100)

biz_new = (calc_new.records.e00900 * calc_new.records.s006).sum()
inc_dict['New'].append(biz_new)
biz = (calc.records.e00900 * calc.records.s006).sum()
inc_dict['Current'].append(biz)
biz_soi = soi_income['BIZ'].sum()
inc_dict['SOI'].append(biz_soi)
new_biz_diff = abs(biz_new - biz_soi)
inc_dict['New - SOI'].append(new_biz_diff)
curr_biz_diff = abs(biz - biz_soi)
inc_dict['Current - SOI'].append(curr_biz_diff)
inc_dict['Pct Diff'].append((curr_biz_diff / biz) * 100)

inc_df = pd.DataFrame.from_dict(inc_dict)
inc_df.index = inc_list
print "'Pct Diff' represents the percentage change between the current and new sums"
inc_df

'Pct Diff' represents the percentage change between the current and new sums


,New,Current,SOI,Pct Diff,New - SOI,Current - SOI
WAS,"6,784,980,284,114.8","6,820,912,371,695.9","6,784,942,966,000.0",0.5,"37,318,114.8","35,969,405,695.9"
Taxable Interest,"93,894,821,368.4","97,246,196,658.4","93,894,281,000.0",3.4,"540,368.4","3,351,915,658.4"
Ordinary Dividends,"254,707,967,594.8","270,137,086,776.2","254,702,232,000.0",5.7,"5,735,594.8","15,434,854,776.2"
Qualified Dividends,"192,856,835,660.6","198,124,994,913.5","192,447,708,000.0",2.9,"409,127,660.6","5,677,286,913.5"
Business Income,"317,256,458,442.1","309,444,223,964.1","317,258,765,000.0",2.5,"2,306,557.9","7,814,541,035.9"


### Distribution of Income Variables

In [151]:
# Generate data for distribution plots
new_dist = pd.DataFrame()
cur_dist = pd.DataFrame()
new_was = distribution(calc_new.records.e00200, calc_new.records.s006, calc_new.records.c00100)
cur_was = distribution(calc.records.e00200, calc.records.s006, calc.records.c00100)
new_int = distribution(calc_new.records.e00300, calc_new.records.s006, calc_new.records.c00100)
cur_int = distribution(calc.records.e00300, calc.records.s006, calc.records.c00100)
new_odiv = distribution(calc_new.records.e00600, calc_new.records.s006, calc_new.records.c00100)
cur_odiv = distribution(calc.records.e00600, calc.records.s006, calc.records.c00100)
new_qdiv = distribution(calc_new.records.e00650, calc_new.records.s006, calc_new.records.c00100)
cur_qdiv = distribution(calc.records.e00650, calc.records.s006, calc.records.c00100)
new_biz = distribution(calc_new.records.e00900, calc_new.records.s006, calc_new.records.c00100)
cur_biz = distribution(calc.records.e00900, calc.records.s006, calc.records.c00100)
new_dist['WAS'] = new_was[1]
cur_dist['WAS'] = cur_was[1]
new_dist['INT'] = new_int[1]
cur_dist['INT'] = cur_int[1]
new_dist['ODIV'] = new_odiv[1]
cur_dist['ODIV'] = cur_odiv[1]
new_dist['QDIV'] = new_qdiv[1]
cur_dist['QDIV'] = cur_qdiv[1]
new_dist['BIZ'] = new_biz[1]
cur_dist['BIZ'] = cur_biz[1]
new_dist['AGI Bin'] = index_list()
cur_dist['AGI Bin'] = index_list()
new_dist['label'] = 'New'
cur_dist['label'] = 'Current'
# Create scatter plot objects
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]
soi_dist = pd.DataFrame()
for item in items_tups:
    soi_dist[item[0]] = (soi_income[item[0]] / soi_income[item[0]].sum()) * 100
soi_dist['AGI Bin'] = index_list()
soi_dist['label'] = 'SOI'
scatter_data = pd.concat([new_dist, cur_dist, soi_dist])
scatter_list = list()  # list for scatter plot objects
for item in items_tups:
    title = 'Percent of Total {} by AGI Bin'.format(item[1])
    scatter = Scatter(scatter_data, x='AGI Bin', y=item[0], color='label', ylabel='Percent',
                      title=title, tooltips=[('PCT', '@{}'.format(item[0]))])
    scatter_list.append(scatter)

In [152]:
show(column(scatter_list))

In [153]:
new_tot = pd.DataFrame()
cur_tot = pd.DataFrame()
new_tot['WAS'] = new_was[0]
cur_tot['WAS'] = cur_was[0]
new_tot['INT'] = new_int[0]
cur_tot['INT'] = cur_int[0]
new_tot['ODIV'] = new_odiv[0]
cur_tot['ODIV'] = cur_odiv[0]
new_tot['QDIV'] = new_qdiv[0]
cur_tot['QDIV'] = cur_qdiv[0]
new_tot['BIZ'] = new_biz[0]
cur_tot['BIZ'] = cur_biz[0]
new_tot['AGI Bin'] = index_list()
cur_tot['AGI Bin'] = index_list()
new_tot['label'] = 'New'
cur_tot['label'] = 'Current'
soi_income['AGI Bin'] = index_list()
soi_income['label'] = 'SOI'
total_data = pd.concat([new_tot, cur_tot, soi_income])
# Create scatter plot objects
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]
total_list = list()  # list for scatter plot objects
for item in items_tups:
    title = 'Total {} by AGI Bin'.format(item[1])
    scatter = Scatter(total_data, x='AGI Bin', y=item[0], color='label', ylabel='Total',
                      title=title, tooltips=[('Total', '@{}'.format(item[0]))])
    total_list.append(scatter)

In [154]:
show(column(total_list))

### Itemized Deduction Amounts

In [155]:
deductions_new = {'Medical Expenses': (calc_new.records.e17500[calc_new.records.c04470 > 0] *
                                        calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'State and Local Taxes':  (calc_new.records.e18400[calc_new.records.c04470 > 0] *
                                              calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'Real Estate Taxes':  (calc_new.records.e18500[calc_new.records.c04470 > 0] *
                                          calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'Interest Paid':  (calc_new.records.e19200[calc_new.records.c04470 > 0] *
                                       calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'Charitable Cash Contributions': (calc_new.records.e19800[calc_new.records.c04470 > 0] *
                                                     calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'Charitable Non-Cash Contributions': (calc_new.records.e20100[calc_new.records.c04470 > 0] *
                                                         calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'Total Misc. Expenses':  (calc_new.records.e20400[calc_new.records.c04470 > 0] *
                                             calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'Net Casualty or Loss': (calc_new.records.g20500[calc_new.records.c04470 > 0] *
                                            calc_new.records.s006[calc_new.records.c04470 > 0]).sum()}
ded_new_df = pd.DataFrame.from_dict(deductions_new, 'index')
ded_new_df.columns = ['Total']
ded_new_df['source'] = 'New'

deductions = {'Medical Expenses': (calc.records.e17500[calc.records.c04470 > 0] *
                                   calc.records.s006[calc.records.c04470 > 0]).sum(),
              'State and Local Taxes':  (calc.records.e18400[calc.records.c04470 > 0] *
                                         calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Real Estate Taxes':  (calc.records.e18500[calc.records.c04470 > 0] *
                                     calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Interest Paid':  (calc.records.e19200[calc.records.c04470 > 0] *
                                 calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Charitable Cash Contributions': (calc.records.e19800[calc.records.c04470 > 0] *
                                                calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Charitable Non-Cash Contributions': (calc.records.e20100[calc.records.c04470 > 0] *
                                                    calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Total Misc. Expenses':  (calc.records.e20400[calc.records.c04470 > 0] *
                                        calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Net Casualty or Loss': (calc.records.g20500[calc.records.c04470 > 0] *
                                       calc.records.s006[calc.records.c04470 > 0]).sum()}
ded_df = pd.DataFrame.from_dict(deductions, 'index')
ded_df.columns = ['Total']
ded_df['source'] = 'Current'
soi_deductions['source'] = 'SOI'

ded_full_df = pd.concat([ded_new_df, ded_df, soi_deductions])
ded_full_df['index'] = ded_full_df.index

In [156]:
ded_bar = Bar(ded_full_df, 'index', 'Total', group='source', title='Itemized Deduction Totals',
              xlabel='Deduction', ylabel='Total', tooltips=[('Deduction', '@index'), ('Total', '@height{0.2f}'),
                                                            ('Data', '@source')])
show(ded_bar)

In [157]:
ded_error_df = pd.DataFrame()
ded_error_df['Difference: New'] = ded_new_df['Total'] - soi_deductions['Total']
ded_error_df['% Difference: New'] = 100 * ded_error_df['Difference: New'] / soi_deductions['Total']
ded_error_df['Difference: Current'] = ded_df['Total'] - soi_deductions['Total']
ded_error_df['% Difference: Current'] = 100 * ded_error_df['Difference: Current'] / soi_deductions['Total']
print 'Error in Itemized Deductions Relative to SOI Totals'
ded_error_df

Error in Itemized Deductions Relative to SOI Totals


,Difference: New,% Difference: New,Difference: Current,% Difference: Current
Charitable Cash Contributions,"-5,943,923,039.0",-3.8,"-4,703,427,583.1",-3.0
Charitable Non-Cash Contributions,"-30,157,917,113.5",-46.2,"-31,555,102,263.5",-48.3
Interest Paid,"60,145,842,472.7",19.5,"-7,489,448,194.8",-2.4
Medical Expenses,"-17,279,164,765.5",-13.4,"-9,779,156,210.6",-7.6
Net Casualty or Loss,"1,757,990,563.6",79.8,"2,251,393,716.6",102.1
Real Estate Taxes,"11,476,968,177.1",6.3,"17,247,882,146.1",9.5
State and Local Taxes,"-18,551,891,517.0",-5.7,"-16,248,434,290.2",-4.9
Total Misc. Expenses,"1,260,913,676.1",1.0,"3,957,677,605.4",3.1


In [158]:
# Create DataFrame subset of calculator records for the new weights
item_df_new = pd.DataFrame({'e00200': calc_new.records.e00200,
                             's006': calc_new.records.s006,
                             'c04470': calc_new.records.c04470})
item_df_new['itemizer'] = np.where(calc_new.records.c04470 > 0, 1, 0)
item_participation_rt_new = percentile(item_df_new, 'itemizer', 100, 'e00200', 's006')
# Create DataFrame subset for current weights
item_df = pd.DataFrame({'e00200': calc.records.e00200,
                        's006': calc.records.s006,
                        'c04470': calc.records.c04470})
item_df['itemizer'] = np.where(calc.records.c04470 > 0, 1, 0)
item_participation_rt = percentile(item_df, 'itemizer', 100, 'e00200', 's006')
item_index = item_participation_rt_new.index

In [159]:
hover_ded = HoverTool(tooltips=[('Percentile', '$index'), ('Itemizing Rate', '$x')])
f = figure(title='Itemizing Rate by Wage Percentile',
           x_axis_label='Wage Percentile', tools=[hover_ded, 'save', 'reset', 'zoom_in', 'zoom_out'])
f.line(item_index, item_participation_rt_new, legend='new')
f.line(item_index, item_participation_rt, legend='Current', color='red')
f.legend.location = 'top_left'
show(f)

In [160]:
# Average Itemized Deduction by percentile
itemizers_new = copy.deepcopy(item_df_new[item_df_new['c04470'] > 0])
item_mean_new = percentile(itemizers_new, 'c04470', 100, 'e00200', 's006')
itemizers = copy.deepcopy(item_df[item_df['c04470'] > 0])
item_mean = percentile(itemizers, 'c04470', 100, 'e00200', 's006')

In [161]:
hover = HoverTool(tooltips=[('Percentile', '$index'), ('Mean', '$y')])
f = figure(title='Mean Itemized Deduction Total Among Itemizers',
           x_axis_label='Wage Percentile', tools=[hover, 'save', 'reset', 'zoom_in', 'zoom_out'])
f.line(item_index, item_mean_new, legend='new')
f.line(item_index, item_mean, legend='Current', color='red')
f.legend.location = 'top_left'
show(f)

### Tax Liability by Year

In [162]:
liabilities_current = {'Income Tax': [],
                       'Payroll Tax': [],
                       'Combined': []}
liabilities_new = {'Income Tax': [],
                   'Payroll Tax': [],
                   'Combined': []}
years = []
for year in range(2015, 2027):
    years.append(year)
    calc.advance_to_year(year)
    calc_new.advance_to_year(year)
    calc.calc_all()
    calc_new.calc_all()
    liabilities_current['Income Tax'].append((calc.records.s006 * calc.records.iitax).sum())
    liabilities_current['Payroll Tax'].append((calc.records.s006 * calc.records.payrolltax).sum())
    liabilities_current['Combined'].append((calc.records.s006 * calc.records.combined).sum())
    liabilities_new['Income Tax'].append((calc_new.records.s006 * calc_new.records.iitax).sum())
    liabilities_new['Payroll Tax'].append((calc_new.records.s006 * calc_new.records.payrolltax).sum())
    liabilities_new['Combined'].append((calc_new.records.s006 * calc_new.records.combined).sum())

In [163]:
f = figure(title='Tax Liabilities 2015-2026')
f.width = 800
f.height = 500
f.line(years, liabilities_current['Income Tax'], color='blue', line_width=2, legend='Income-Current')
f.line(years, liabilities_new['Income Tax'], color='cyan', line_width=2, legend='Income-New')
f.line(years, liabilities_current['Payroll Tax'], color='purple', line_width=2, legend='Payroll-Current')
f.line(years, liabilities_new['Payroll Tax'], color='magenta', line_width=2, legend='Payroll-New')
f.line(years, liabilities_current['Combined'], color='green', line_width=2, legend='Combined-Current')
f.line(years, liabilities_new['Combined'], color='lime', line_width=2, legend='Combined-New')
f.legend.location = 'top_left'
f.xaxis[0].ticker.desired_num_ticks = 12
show(f)

In [164]:
ldf_current = pd.DataFrame(liabilities_current, index=years)
print 'Current Tax Liabilities by Year'
ldf_current

Current Tax Liabilities by Year


,Combined,Income Tax,Payroll Tax
2015,"2,531,299,998,039.8","1,533,690,884,976.4","997,609,113,063.4"
2016,"2,661,259,683,910.4","1,623,615,602,225.5","1,037,644,081,684.8"
2017,"2,790,644,496,968.0","1,700,180,840,249.3","1,090,463,656,718.6"
2018,"2,899,845,848,826.4","1,763,812,025,892.1","1,136,033,822,934.4"
2019,"2,999,153,205,160.1","1,820,779,794,762.7","1,178,373,410,397.3"
2020,"3,105,294,507,042.3","1,884,126,754,413.6","1,221,167,752,628.8"
2021,"3,230,472,254,858.9","1,962,352,941,180.3","1,268,119,313,678.6"
2022,"3,369,209,819,645.6","2,051,077,600,598.9","1,318,132,219,046.8"
2023,"3,518,927,703,011.7","2,148,265,187,661.9","1,370,662,515,349.9"
2024,"3,676,852,849,876.3","2,251,436,564,251.5","1,425,416,285,624.7"


In [165]:
ldf_new = pd.DataFrame(liabilities_new, index=years)
print 'New Tax Liabilities by Year'
ldf_new

New Tax Liabilities by Year


,Combined,Income Tax,Payroll Tax
2015,"2,423,816,933,426.3","1,414,898,293,298.0","1,008,918,640,128.3"
2016,"2,548,290,170,885.9","1,492,421,165,410.0","1,055,869,005,476.0"
2017,"2,652,214,153,610.4","1,548,171,817,587.7","1,104,042,336,022.7"
2018,"2,762,601,197,205.8","1,612,981,211,890.6","1,149,619,985,315.2"
2019,"2,848,496,913,084.5","1,654,465,760,788.5","1,194,031,152,296.0"
2020,"2,953,477,903,098.6","1,717,049,454,770.0","1,236,428,448,328.6"
2021,"3,071,714,709,985.6","1,787,484,809,791.9","1,284,229,900,193.7"
2022,"3,204,077,948,434.3","1,869,464,167,206.1","1,334,613,781,228.2"
2023,"3,343,206,611,806.6","1,955,083,798,429.7","1,388,122,813,376.8"
2024,"3,491,266,389,382.7","2,047,403,898,689.7","1,443,862,490,693.1"


In [166]:
print 'Difference in Tax Liabilities by Year'
ldf_new - ldf_current

Difference in Tax Liabilities by Year


,Combined,Income Tax,Payroll Tax
2015,"-107,483,064,613.5","-118,792,591,678.5","11,309,527,064.9"
2016,"-112,969,513,024.4","-131,194,436,815.6","18,224,923,791.1"
2017,"-138,430,343,357.6","-152,009,022,661.6","13,578,679,304.0"
2018,"-137,244,651,620.6","-150,830,814,001.5","13,586,162,380.8"
2019,"-150,656,292,075.6","-166,314,033,974.2","15,657,741,898.7"
2020,"-151,816,603,943.7","-167,077,299,643.6","15,260,695,699.8"
2021,"-158,757,544,873.2","-174,868,131,388.4","16,110,586,515.1"
2022,"-165,131,871,211.3","-181,613,433,392.8","16,481,562,181.5"
2023,"-175,721,091,205.2","-193,181,389,232.2","17,460,298,027.0"
2024,"-185,586,460,493.5","-204,032,665,561.9","18,446,205,068.3"


In [167]:
print 'Pct. Change in Tax Liabilities by Year'
((ldf_new - ldf_current) / ldf_current) * 100

Pct. Change in Tax Liabilities by Year


,Combined,Income Tax,Payroll Tax
2015,-4.2,-7.7,1.1
2016,-4.2,-8.1,1.8
2017,-5.0,-8.9,1.2
2018,-4.7,-8.6,1.2
2019,-5.0,-9.1,1.3
2020,-4.9,-8.9,1.2
2021,-4.9,-8.9,1.3
2022,-4.9,-8.9,1.3
2023,-5.0,-9.0,1.3
2024,-5.0,-9.1,1.3
